In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("samaxr/code-summary-java")
print(dataset)
train_data = dataset["train"]
subset = train_data.select(range(10000))  # Select first 200 indices

# for item in subset:
#     code = item["code"]
#     summary = item["summary"]
    # ... (process code and summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/285670 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/31741 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/79352 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'summary'],
        num_rows: 285670
    })
    validation: Dataset({
        features: ['code', 'summary'],
        num_rows: 31741
    })
    test: Dataset({
        features: ['code', 'summary'],
        num_rows: 79352
    })
})


In [ ]:
from transformers import RobertaTokenizer
import concurrent.futures

tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base')

# Define function to preprocess a batch of data
def preprocess_batch(item):
    processed_batch = []
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    inputs = tokenizer.encode_plus(summary, code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length', return_tensors="pt")

    # Add processed data to list
    processed_batch.append({
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask']
    })

    return processed_batch


# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Divide the data into batches
        batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

        # Preprocess each batch using multiple processes
        for processed_batch in executor.map(preprocess_batch, batches):
            processed_data.extend(processed_batch)
    return processed_data


# Set batch size
batch_size = 16

# Assuming you have the 'train_data' variable containing your training data
# Preprocess data in batches
processed_data = preprocess_data_multiprocessed(train_data, batch_size)

print("Processed")

# Now you can use 'processed_data' to train your CodeBERT model



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Processed


In [ ]:
# Install necessary libraries
!pip install transformers
!pip install datasets

# Import required libraries
from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
import torch
import concurrent.futures
from tqdm import tqdm
import os

# Load the CodeBERT tokenizer
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Load the dataset
dataset = load_dataset("samaxr/code-summary-java")
train_data = dataset["train"]
subset = train_data.select(range(100))  # Select first 50 indices

# Define your dataset class
class CodeSummaryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define your model class
class CodeSummaryModel(torch.nn.Module):
    def __init__(self):
        super(CodeSummaryModel, self).__init__()
        self.codebert = RobertaModel.from_pretrained("microsoft/codebert-base")
        self.fc = torch.nn.Linear(self.codebert.config.hidden_size, 2)  # Assuming binary classification, adjust output size accordingly

    def forward(self, input_ids, attention_mask):
        outputs = self.codebert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Define function to preprocess a batch of data
def preprocess_batch(item):
    processed_batch = []
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    # Add processed data to list
    processed_batch.append({
        'input_ids': code_tokens,
        'attention_mask': [1] * len(code_tokens),
        'labels': 0,  # Assuming this is for binary classification
        'summary_ids': summary_tokens,
        'summary_mask': [1] * len(summary_tokens)
    })

    return processed_batch

# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Divide the data into batches
        batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

        # Preprocess each batch using multiple processes
        for processed_batch in executor.map(preprocess_batch, batches):
            processed_data.extend(processed_batch)
    return processed_data

# Set batch size
batch_size = 16

# Preprocess data in batches
processed_data = preprocess_data_multiprocessed(subset, batch_size)

# Convert preprocessed data to DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.data[idx].items()}

custom_dataset = CustomDataset(processed_data)
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the number of epochs
epochs = 10  # Increased to 10 epochs

# Instantiate your model
model = CodeSummaryModel()
model.to(device)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Define your training loop
def train(model, train_dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = torch.nn.functional.cross_entropy(logits.view(-1, 2), labels)  # Assuming binary classification
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    return avg_train_loss

# Train your model
for epoch in range(epochs):
    avg_train_loss = train(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")


# Specify the directory to save the model
save_dir = '/content/model_directory'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model's state dictionary
torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))




/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_depreca

Epoch 1/10, Average Training Loss: 0.8862


Training: 100%|██████████| 1/1 [00:53<00:00, 53.13s/it]


Epoch 2/10, Average Training Loss: 0.6889


Training: 100%|██████████| 1/1 [00:53<00:00, 53.41s/it]


...

Epoch 8/10, Average Training Loss: 0.1961


Training: 100%|██████████| 1/1 [00:45<00:00, 45.80s/it]


Epoch 9/10, Average Training Loss: 0.1480


Training: 100%|██████████| 1/1 [00:44<00:00, 44.78s/it]


Epoch 10/10, Average Training Loss: 0.1491


In [ ]:
def generate_summary(input_code):
    # Tokenize the input code
    input_tokens = tokenizer.encode(input_code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')

    # Convert tokens to tensor
    input_ids = torch.tensor(input_tokens).unsqueeze(0)  # Add batch dimension
    attention_mask = torch.ones_like(input_ids)  # Assuming no padding

    # Generate summary
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state

    # Decode the summary tokens
    generated_summary = tokenizer.decode(last_hidden_state[0], skip_special_tokens=True)

    # Return the generated summary
    return generated_summary




In [ ]:
input_code = """
public class Main {
    public static void main(String[] args) {
        System.out.println("Hello, world!");
    }
}
"""

generated_summary = generate_summary(input_code)
print("Generated Summary:", generated_summary)



AttributeError: 'Tensor' object has no attribute 'last_hidden_state'

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=042ca013e666626a4df1a217e5175bb3ccb5528d9599f9fc865195f49a22792e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
from tqdm import tqdm

# Define a function to generate summaries for test data
def generate_summaries(model, test_dataloader, device):
    model.eval()
    generated_summaries = []
    ground_truth_summaries = []

    for batch in tqdm(test_dataloader, desc="Generating Summaries"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        with torch.no_grad():
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            predicted_summary_ids = torch.argmax(logits, dim=-1)

        # Decode predicted summaries
        decoded_summaries = [tokenizer.decode(summary_ids, skip_special_tokens=True) for summary_ids in predicted_summary_ids]
        generated_summaries.extend(decoded_summaries)

        # Gather ground truth summaries
        ground_truth_summaries.extend([tokenizer.decode(summary_ids, skip_special_tokens=True) for summary_ids in batch['summary_ids']])

    return generated_summaries, ground_truth_summaries

# Generate summaries for test data
test_data = dataset["test"]
test_subset = test_data.select(range(50))  # Select first 50 indices for testing
test_processed_data = preprocess_data_multiprocessed(test_subset, batch_size)
test_custom_dataset = CustomDataset(test_processed_data)
test_dataloader = DataLoader(test_custom_dataset, batch_size=batch_size)
generated_summaries, ground_truth_summaries = generate_summaries(model, test_dataloader, device)

# Calculate ROUGE-L score
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
total_rouge_l_score = 0.0

for generated_summary, ground_truth_summary in zip(generated_summaries, ground_truth_summaries):
    rouge_l_score = scorer.score(generated_summary, ground_truth_summary)['rougeL'].fmeasure
    print("Generated summary:", generated_summary)
    print("Ground truth summary:", ground_truth_summary)
    print("rouge_l_score intermediate result = ", rouge_l_score)
    total_rouge_l_score += rouge_l_score

avg_rouge_l_score = total_rouge_l_score / len(generated_summaries)
print(f"Average ROUGE-L Score: {avg_rouge_l_score:.4f}")


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Generating Summaries: 100%|██████████| 1/1 [00:13<00:00, 13.95s/it]

Generated summary: <s>
Ground truth summary: 
rouge_l_score intermediate result =  0
Generated summary: <s>
Ground truth summary: 
rouge_l_score intermediate result =  0
Generated summary: <s>
Ground truth summary: 
rouge_l_score intermediate result =  0
Generated summary: <s>
Ground truth summary: 
rouge_l_score intermediate result =  0
Average ROUGE-L Score: 0.0000
